In this notebook we add carbon biosphere flows to the IPCC 2013 method so that we can have negative CO2 emissions from biomass CCS

import useful libraries and set current project:

In [ ]:
%run initialize_notebook.ipynb

# Create IPCC 2013 method will all non-fossil CO2 characterised:

In [ ]:
ipcc = Method(('IPCC 2013', 'climate change', 'GWP 100a'))
gwp_data = ipcc.load()

find non-fossil CO2 biosphere flows:

In [ ]:
non_fossil = [x for x in get_many(Database("biosphere3"), equals("name", "Carbon dioxide, non-fossil"))]
non_fossil

In [ ]:
gwp_data.extend([(x.key, 1.) for x in non_fossil])

Find CO2 in air biosphere flow

In [ ]:
co2_in_air = get_one(Database("biosphere3"), equals("name", 'Carbon dioxide, in air'))

In [ ]:
co2_in_air

In [ ]:
gwp_data.append((co2_in_air.key, -1.))

In [ ]:
round(pd.Series({get_activity(x)['name']: y for x, y in gwp_data}),2)

In [ ]:
method = Method(('IPCC 2013', 'climate change', 'GWP 100a', 'Complete'))
method.register()
method.write(gwp_data)
method.process()

# Create a new biosphere flow for non-fossil CO2 to geological storage.

In [ ]:
biosphere = Database('biosphere3')
biosphere

In [ ]:
new_flow = biosphere.new_activity('CO2 to geological storage, non-fossil', **{'name': 'CO2 to geological storage, non-fossil',
                                                                                'unit': 'kilogram',
                                                                                'type': 'storage',
                                                                                'categories': ('geological storage',)})

new_flow.save()

In [ ]:
co2_to_storage = [x for x in Database("biosphere3") if 'CO2 to geological storage, non-fossil' in x['name']][0]

co2_to_storage

Step 2: add characterization for this flow - we will copy a flow for non-fossil CO2 to soil or biomass stock, as it will have the same characterisation factors. 

In [ ]:
co2_to_soil = [x for x in Database("biosphere3") if "Carbon dioxide, to soil or biomass stock" in str(x) and "('soil',))" in str(x)][0]
co2_to_soil

Here we have a list of methods that we want to add the flow to:


In [ ]:
lcia_methods={
    'CC':('IPCC 2013', 'climate change', 'GWP 100a'),
    'TA':('ReCiPe Midpoint (H)', 'terrestrial acidification', 'TAP100'),
    'POF':('ReCiPe Midpoint (H)','photochemical oxidant formation','POFP'),
    'PMF':('ReCiPe Midpoint (H)', 'particulate matter formation', 'PMFP'),
    'MD':('ReCiPe Midpoint (H)', 'metal depletion', 'MDP'),
    'HT':('ReCiPe Midpoint (H)', 'human toxicity', 'HTPinf'),
    'MET':('ReCiPe Midpoint (H)', 'marine ecotoxicity', 'METPinf'),
    'ME':('ReCiPe Midpoint (H)', 'marine eutrophication', 'MEP'),
    'FD':('ReCiPe Midpoint (H)', 'fossil depletion', 'FDP'),
    'IR':('ReCiPe Midpoint (H)', 'ionising radiation', 'IRP_HE'),
    'OD':('ReCiPe Midpoint (H)', 'ozone depletion', 'ODPinf'),
    'FET':('ReCiPe Midpoint (H)', 'freshwater ecotoxicity', 'FETPinf'),
    'TET':('ReCiPe Midpoint (H)', 'terrestrial ecotoxicity', 'TETPinf'),
    'ALO':('ReCiPe Midpoint (H)', 'agricultural land occupation', 'ALOP'),
    'NLT':('ReCiPe Midpoint (H)', 'natural land transformation', 'NLTP'),
    'ULO':('ReCiPe Midpoint (H)', 'urban land occupation', 'ULOP'),
    'WD':('ReCiPe Midpoint (H)', 'water depletion', 'WDP'),
    'FE':('ReCiPe Midpoint (H)', 'freshwater eutrophication', 'FEP'),
    'R_HH' : ('ReCiPe Endpoint (H,A)', 'human health', 'total'),
    'R_EQ' : ('ReCiPe Endpoint (H,A)', 'ecosystem quality', 'total'),
    'R_R' : ('ReCiPe Endpoint (H,A)', 'resources', 'total'),
    'R_Total' : ('ReCiPe Endpoint (H,A)', 'total', 'total'),
    
    'CEDB': ('cumulative energy demand','biomass','renewable energy resources, biomass'),
      'CEDF': ('cumulative energy demand','fossil','non-renewable energy resources, fossil'),
      'CEDG': ('cumulative energy demand','geothermal','renewable energy resources, geothermal, converted'),
      'CEDN': ('cumulative energy demand','nuclear','non-renewable energy resources, nuclear'),
      'CEDFr': ('cumulative energy demand','primary forest','non-renewable energy resources, primary forest'),
      'CEDS': ('cumulative energy demand','solar','renewable energy resources, solar, converted'),
      'CEDH': ('cumulative energy demand','water','renewable energy resources, potential (in barrage water), converted'),
      'CEDW': ('cumulative energy demand','wind','renewable energy resources, kinetic (in wind), converted')

    }

In [ ]:
for cat in lcia_methods:
    method  = Method(lcia_methods[cat])
    method_data = method.load()
    #first make sure we don't already have the flow included:
    if [x for x in method_data if co2_to_storage.key[1] in x[0][1]]:
        print('Flow already present- you must have run this code already.')
        continue
    
    else:
        try:
            characterized_flow = [x for x in method_data if co2_to_soil.key[1] in x[0][1]][0]
        except:
            continue
        
        method_data.extend([(co2_to_storage.key, characterized_flow[1]) ])
            
        print('Flow added to method: {}'.format(method.name))
        print('Characterisation factor: {}'.format(characterized_flow[1]))
        
        orig_name = [x for x in method.name]
        new_method = Method(tuple(orig_name +['CO2 storage']))
        new_method.register()
        new_method.write(method_data)
        new_method.process()        

In [ ]:
new_methods = [x for x in methods if 'CO2 storage' in str(x)]
new_methods

In [ ]:
Method(new_methods[0]).load()[10:]